In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

In [ ]:
class WeatherTransformer(nn.Module):
    def __init__(self, feature_size, num_layers=1, num_decoders=5):
        super(WeatherTransformer, self).__init__()
        self.num_decoders = num_decoders

        encoder_layers = TransformerEncoderLayer(d_model=feature_size, nhead=5)
        self.transformer_encoder = TransformerEncoder(encoder_layers, num_layers)

        self.decoders = nn.ModuleList([
            TransformerDecoder(TransformerDecoderLayer(d_model=feature_size, nhead=5), num_layers)
            for _ in range(num_decoders)
        ])
    
    def forward(self, src):
        memory = self.transformer_encoder(src)
        outputs = []
        for decoder in self.decoders:
            output = decoder(memory, memory)
            outputs.append(output)
        return outputs

In [ ]:

def load_data(files):
    data = [pd.read_excel(file) for file in files]
    return data

def prepare_data(data, labels=True):
    if labels:

        features = [torch.tensor(d.iloc[:, 1:].values, dtype=torch.float32) for d in data]
        targets = [torch.tensor(d.iloc[:, 0].values, dtype=torch.float32).view(-1, 1) for d in data]
        return features, targets
    else:
        features = [torch.tensor(d.values, dtype=torch.float32) for d in data]
        return features
    

def train_model(model, train_loader, num_epochs=2):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for features, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(features)
            loss = sum([criterion(output.squeeze(), target) for output, target in zip(outputs, targets)])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}')


def predict(model, test_data):
    model.eval()
    predictions = []
    with torch.no_grad():
        for data in test_data:
            output = model(data.unsqueeze(0)) 
            predictions.append(output)
    return predictions

In [ ]:

if __name__ == "__main__":
    train_files = ['train_1.xlsx', 'train_2.xlsx', 'train_3.xlsx',
                   ]
    test_files = ['test_1.xlsx', 'test_2.xlsx', 'test_3.xlsx',
                  ]


    train_data = load_data(train_files)
    train_features, train_targets = prepare_data(train_data)
    train_dataset = TensorDataset(torch.cat(train_features, dim=0), torch.cat(train_targets, dim=0))
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


    feature_size = 5 
    model = WeatherTransformer(feature_size)


    train_model(model, train_loader)

    test_data = load_data(test_files)
    test_features = prepare_data(test_data, labels=False)


    predictions = predict(model, test_features)


In [ ]:

all_predictions = pd.DataFrame()


for idx, pred in enumerate(predictions):
    pred_df = pd.DataFrame(pred[0].squeeze().numpy())
    pred_df.columns = [f'prediction_wind_farm_{idx+1}_col_{i+1}' for i in range(pred_df.shape[1])] 
    if all_predictions.empty:
        all_predictions = pred_df
    else:
        all_predictions = pd.concat([all_predictions, pred_df], axis=1)


all_predictions.to_csv('all_predictions.csv', index=False)
